In [1]:
# Load modules necessary
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from scipy.signal import detrend

%matplotlib qt

## Load sample dataset

##### For EEG dataset, we will use EEG Motor Movement/Imagery Dataset from here: https://physionet.org/content/eegmmidb/1.0.0/#files-panel

##### The original dataset contains 109 volunteers. However, here in this tutorial we will explore data from Subject 42
##### You can download the dataset from Brightspace

In [2]:
# Path to the EEG file
eegPath = '../../Datasets/EEG/S042/S042R01.edf'

# Load the EEG file using MNE
# MNE has different read formats for different EEG file types
# Here we are using read_raw_edf to read the EEG file
# preload=True loads the data into memory (default is False, which loads the data when needed)
raw = mne.io.read_raw_edf(eegPath, preload=True)

Extracting EDF parameters from /Users/jiahuan/Desktop/NYU/PSYCH-GA 3405 EEG/My_Fork/Datasets/EEG/S042/S042R01.edf...
EDF file detected


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jiahuan/Desktop/NYU/PSYCH-GA 3405 EEG/My_Fork/Datasets/EEG/S042/S042R01.edf'

In [3]:
# Let us try plotting an EEG channel using matplotlib
# We can get the data of the EEG channel using get_data() function
# The data is in the form of a numpy array
data = raw.get_data()
plt.plot(raw.times, data[0, :])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')
plt.title('EEG Channel 1')
plt.show()

NameError: name 'raw' is not defined

In [ ]:
# Now let's try using the interactive plotting feature of MNE
# This is done by using the plot() function of the raw object
# This will open a new window where we can interactively plot the EEG data
# This is useful for exploring the data, checking for any artifacts, finding bad channels, or sections of recordings that need to be removed
raw.plot(scalings=10e-5, n_channels=8, title='EEG Data')
plt.show()

In [ ]:
# Referencing data
# EEG data is usually recorded with respect to a reference electrode
# The reference electrode can be different for different EEG systems
# The data can be re-referenced to a common reference electrode, such as the average reference, linked mastoids, etc.
# Here we are going to re-reference the data using the average reference and the mastoids as reference electrodes
# The average refernce is calculated by taking the average of all the electrodes
raw_reref = raw.copy() # Create a copy of the raw object so that the original data is not modified
raw_reref.set_eeg_reference(ref_channels='average')
# We can also rerefence data using a specific channel as the reference electrode
# For example, we can use the mastoids as reference electrodes
raw_reref2 = raw.copy()
raw_reref2.set_eeg_reference(ref_channels= ['T7..', 'T8..'])

# Plot the data before referencing
fig1 = raw.plot(scalings=10e-5, n_channels=8, title='EEG Data (Original)', block=False)
# Plot the data after referencing using average reference
fig2 = raw_reref.plot(scalings=10e-5, n_channels=8, title='EEG Data (Average Reference)', block=False)
# Plot the data after referencing using first channel as reference
fig3 = raw_reref2.plot(scalings=10e-5, n_channels=8, title='EEG Data (Mastoid Reference)', block=False)
plt.show()

In [ ]:
# Creating a fake channel with a polynomial trend
fake_data = raw.get_data()[0]
t = np.linspace(0, 1, len(fake_data))
np.random.seed(42)
trend = np.random.randn(3)
fake_data += 4e-3 * np.polyval(trend, t)

# Let us look at different ways to remove trend from the data
# First, we can simply mean-center the data
# This is done by subtracting the mean of the data from the data
fake_data_centered = fake_data - np.mean(fake_data)

# Second, we can remove a linear trend from the data
# This can be done using detrend() function of numpy
fake_data_detrended = detrend(fake_data, type='linear')

# Next we can remove a polynomial trend from the data
# This is done by fitting a polynomial model to the data and subtracting the model from the data
# We can use the polyfit() function of numpy to fit a polynomial model to the data
# The polyfit() function returns the coefficients of the polynomial model
# We can use the polyval() function of numpy to evaluate the polynomial model at the data points
# We can then subtract the polynomial model from the data
trend = np.polyfit(t, fake_data, 3)
fake_data_detrended2 = fake_data - np.polyval(trend, t)

# Plot the original data and the detrended data
f, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].plot(t, fake_data)
axs[0, 0].set_title('Original Data')
axs[0, 1].plot(t, fake_data_centered)
axs[0, 1].set_title('Mean-Centered Data')
axs[1, 0].plot(t, fake_data_detrended)
axs[1, 0].set_title('Linear Detrended Data')
axs[1, 1].plot(t, fake_data_detrended2)
axs[1, 1].set_title('Polynomial Detrended Data')
plt.show()